# Spaceship Titanic – Data Cleaning, Imputation, Feature Engineering, and Scaling

Process:
- Load raw `train.csv` and `test.csv`.
- Concat train and test for consistent transformations.
- Feature engineering:
  - Split `PassengerId` into `PassengerGroup` and `PassengerNumber`.
  - Split `Cabin` into `CabinDeck`, `CabinNum`, and `CabinSide`.
  - Split `Name` into `GivenNames` and `Surname`.
  - Aggregate spending features and log-transform them.
- Add **missing flags** for important columns.
- Do data imputation.
- Onehot encode categorical features.
- Scale numeric/log transformed features.
- Split back into cleaned train and test sets.
- Save as `train_clean.csv` and `test_clean.csv`.

In [10]:

import os
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Try typical Kaggle path first, then fall back to local/project paths
POSSIBLE_DIRS = [
    "/kaggle/input/spaceship-titanic",
    "from_kaggle",
]

def find_data_dir():
    for d in POSSIBLE_DIRS:
        train_candidate = os.path.join(d, "train.csv")
        test_candidate = os.path.join(d, "test.csv")
        if os.path.exists(train_candidate) and os.path.exists(test_candidate):
            return d
    raise FileNotFoundError("Could not find train.csv and test.csv in known locations.")

DATA_DIR = find_data_dir()
print("Using data directory:", DATA_DIR)

train_path = os.path.join(DATA_DIR, "train.csv")
test_path = os.path.join(DATA_DIR, "test.csv")

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

print("Train shape:", train.shape)
print("Test shape :", test.shape)
train.head()


Using data directory: from_kaggle
Train shape: (8693, 14)
Test shape : (4277, 13)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Concatenate Train and Test
Add markers so we can apply identical transformations to both sets and then split them back later

In [11]:
target_col = "Transported"

# mark train vs test and make sure target is present in both
train["is_train"] = 1
test["is_train"] = 0

# add placeholder target for test to keep columns aligned
if target_col not in test.columns:
    test[target_col] = np.nan

full = pd.concat([train, test], ignore_index=True)
print("Full concatenated shape:", full.shape)
full.head()


Full concatenated shape: (12970, 15)


/var/folders/hd/6z2wjwxj7qg9h_22989lv6d40000gn/T/ipykernel_33504/3910981746.py:11: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  full = pd.concat([train, test], ignore_index=True)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,is_train
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0,1
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0,1
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0,1


# Feature Engineering – Structured Columns
Redo the feature engineering logic from the EDA notebook

In [12]:
def split_passenger_id(df):
    pid = df["PassengerId"].str.split("_", expand=True)
    df["PassengerGroup"] = pid[0]
    df["PassengerNumber"] = pd.to_numeric(pid[1], errors="coerce")

def split_cabin(df):
    cabin_parts = df["Cabin"].str.split("/", expand=True)
    df["CabinDeck"] = cabin_parts[0]
    df["CabinNum"] = pd.to_numeric(cabin_parts[1], errors="coerce")
    df["CabinSide"] = cabin_parts[2]

def split_name(df):
    names = df["Name"].fillna("Unknown Unknown")
    name_parts = names.str.rsplit(" ", n=1, expand=True)
    df["GivenNames"] = name_parts[0]
    df["Surname"] = name_parts[1]

split_passenger_id(full)
split_cabin(full)
split_name(full)

full[["PassengerId", "PassengerGroup", "PassengerNumber", "Cabin", "CabinDeck", "CabinNum", "CabinSide", "Name", "GivenNames", "Surname"]].head()


,PassengerId,PassengerGroup,PassengerNumber,Cabin,CabinDeck,CabinNum,CabinSide,Name,GivenNames,Surname
0,0001_01,0001,1,B/0/P,B,0.0,P,Maham Ofracculy,Maham,Ofracculy
1,0002_01,0002,1,F/0/S,F,0.0,S,Juanna Vines,Juanna,Vines
2,0003_01,0003,1,A/0/S,A,0.0,S,Altark Susent,Altark,Susent
3,0003_02,0003,2,A/0/S,A,0.0,S,Solam Susent,Solam,Susent
4,0004_01,0004,1,F/1/S,F,1.0,S,Willy Santantines,Willy,Santantines


# Spending Features and Log Scaling
Aggregate the spending features and log transform them to reduce skew.

In [13]:
spend_cols = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]

# total spend
full["TotalSpend"] = full[spend_cols].sum(axis=1)

# log transform spending columns including TotalSpend
for col in spend_cols + ["TotalSpend"]:
    full[f"{col}_log"] = np.log1p(full[col])  # log(1 + x) to avoid zero issues

full[[*spend_cols, "TotalSpend", "TotalSpend_log"]].head()


,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,TotalSpend,TotalSpend_log
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,109.0,9.0,25.0,549.0,44.0,736.0,6.602588
2,43.0,3576.0,0.0,6715.0,49.0,10383.0,9.248021
3,0.0,1283.0,371.0,3329.0,193.0,5176.0,8.551981
4,303.0,70.0,151.0,565.0,2.0,1091.0,6.995766


# Missing Flags
Add indicators that flag if important features were originally missing

In [14]:
important_missing_cols = [
    "HomePlanet",
    "CryoSleep",
    "Cabin",
    "Destination",
    "Age",
] + spend_cols

for col in important_missing_cols:
    flag_col = f"{col}_was_missing"
    full[flag_col] = full[col].isna().astype(int)

missing_flag_cols = [c for c in full.columns if c.endswith("_was_missing")]
print("Missingness flags created:", len(missing_flag_cols))
full[missing_flag_cols].head()


Missingness flags created: 10


,HomePlanet_was_missing,CryoSleep_was_missing,Cabin_was_missing,Destination_was_missing,Age_was_missing,RoomService_was_missing,FoodCourt_was_missing,ShoppingMall_was_missing,Spa_was_missing,VRDeck_was_missing
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0


# Imputation Strategy
Use a simple strategy:
- **Categorical columns**: fill with the mode.
- **Numeric columns**: fill with the median.

Can go back and use **KNNImputer** later

In [15]:
cat_cols = full.select_dtypes(include="object").columns.tolist()
num_cols = full.select_dtypes(include=["int64", "float64"]).columns.tolist()

# do not impute is_train flag and target via numeric logic
num_cols = [c for c in num_cols if c not in ["is_train"]]

print("Number of categorical columns:", len(cat_cols))
print("Number of numeric columns:", len(num_cols))

# categorical
for col in cat_cols:
    mode_val = full[col].mode(dropna=True)[0]
    full[col] = full[col].fillna(mode_val)

# numeric
use_knn_for_numeric = False 

if use_knn_for_numeric:
    imputer = KNNImputer(n_neighbors=5)
    full[num_cols] = imputer.fit_transform(full[num_cols])
else:
    for col in num_cols:
        median_val = full[col].median()
        full[col] = full[col].fillna(median_val)

print("Total remaining missing values after imputation:", full.isna().sum().sum())


Number of categorical columns: 12
Number of numeric columns: 26
Total remaining missing values after imputation: 0


# One-Hot Encoding of Categorical Variables
Onehot encode all categorical columns. Don't drop the first level so models can decide how to use all categories.

In [16]:
full_encoded = pd.get_dummies(full, columns=cat_cols, drop_first=False)
print("Shape after one-hot encoding:", full_encoded.shape)
full_encoded.head()


Shape after one-hot encoding: (12970, 50042)


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,is_train,PassengerNumber,CabinNum,...,Surname_Wooterston,Surname_Workmans,Surname_Workmanson,Surname_Wrempeedly,Surname_Wriggins,Surname_Wynneyerson,Surname_Yanton,Surname_Yatters,Surname_Yorkland,Surname_Youngrayes
0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0
1,24.0,109.0,9.0,25.0,549.0,44.0,1.0,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0.0,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0.0,1,2,0.0,...,0,0,0,0,0,0,0,0,0,0
4,16.0,303.0,70.0,151.0,565.0,2.0,1.0,1,1,1.0,...,0,0,0,0,0,0,0,0,0,0


# Scaling Numeric Features
Standardize the log transformed and other numeric features to have mean 0 and variance 1

In [17]:
scaler = StandardScaler()

numeric_for_scaling = []
for c in full_encoded.columns:
    if c in num_cols and c not in ["is_train", target_col]:
        numeric_for_scaling.append(c)
    elif c.endswith("_log"):
        numeric_for_scaling.append(c)

print("Number of features to scale:", len(numeric_for_scaling))

full_encoded[numeric_for_scaling] = scaler.fit_transform(full_encoded[numeric_for_scaling])
full_encoded.head()


Number of features to scale: 25


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,is_train,PassengerNumber,CabinNum,...,Surname_Wooterston,Surname_Workmans,Surname_Workmanson,Surname_Wrempeedly,Surname_Wriggins,Surname_Wynneyerson,Surname_Yanton,Surname_Yatters,Surname_Yorkland,Surname_Youngrayes
0,0.720932,-0.340290,-0.281822,-0.292365,-0.269707,-0.257100,0.0,1,-0.490655,-1.181321,...,0,0,0,0,0,0,0,0,0,0
1,-0.332557,-0.170439,-0.276082,-0.249566,0.221040,-0.219449,1.0,1,-0.490655,-1.181321,...,0,0,0,0,0,0,0,0,0,0
2,2.055350,-0.273285,1.998823,-0.292365,5.732776,-0.215170,0.0,1,-0.490655,-1.181321,...,0,0,0,0,0,0,0,0,0,0
3,0.299536,-0.340290,0.536429,0.342766,2.706059,-0.091947,0.0,1,0.468615,-1.181321,...,0,0,0,0,0,0,0,0,0,0
4,-0.894417,0.131863,-0.237179,-0.033861,0.235342,-0.255389,1.0,1,-0.490655,-1.179351,...,0,0,0,0,0,0,0,0,0,0


# Split Back into Train and Test, Save Cleaned Files
**do not overwrite** the original `train.csv` and `test.csv`

In [ ]:
train_clean = full_encoded[full_encoded["is_train"] == 1].copy()
test_clean = full_encoded[full_encoded["is_train"] == 0].copy()

# drop helper flags
train_clean.drop(columns=["is_train"], inplace=True)
test_clean.drop(columns=["is_train"], inplace=True)

# Make sure target is not present in test_clean
if target_col in test_clean.columns:
    test_clean.drop(columns=[target_col], inplace=True)

print("Train_clean shape:", train_clean.shape)
print("Test_clean shape :", test_clean.shape)

train_clean_path = os.path.join(DATA_DIR, "train_clean.csv")
test_clean_path = os.path.join(DATA_DIR, "test_clean.csv")

train_clean.to_csv(train_clean_path, index=False)
test_clean.to_csv(test_clean_path, index=False)

print("Saved train_clean to:", train_clean_path)
print("Saved test_clean  to:", test_clean_path)
